In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClusterMixin
import itertools

In [2]:
def standardize(labels):
#Standardize numerical feature
    return (labels-labels.mean())/labels.std(ddof=0)

def cat_encode(labels):
    ids, new_labels = np.unique(labels, return_inverse=True)
    return new_labels, ids
    
def mode2(a, axis=0, val=False):
#Return mode or mode freq of an array
#code based on scipy.stats.mode
    scores = np.unique(np.ravel(a))      
    testshape = list(a.shape)
    testshape[axis] = 1
    oldmostfreq = np.zeros(testshape, dtype=a.dtype)
    oldcounts = np.zeros(testshape, dtype=int)

    for score in scores:
        template = (a == score)
        counts = np.expand_dims(np.sum(template, axis),axis)
        mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)
        oldcounts = np.maximum(counts, oldcounts)
        oldmostfreq = mostfrequent
    
    return oldcounts[0] if val else mostfrequent[0]


In [4]:
a = pd.read_csv('data/3nja-bsch.tsv', sep='\t')
a = a.round(1)
#a2 = a.dropna().copy()
a2 = a.copy()
for col in a2:
    typ = str(a2[col].dtype)
    if typ in ['int', 'int32', 'int64','float', 'float32', 'float64']:
        a2[col] = standardize(a2[col])
   

a3 = a2.copy().dropna()
a3.reset_index(inplace=True, drop=True)

In [97]:
b = pd.read_csv('data/h3zm-ta5h.tsv', sep='\t')
b = b.round(1)
b2 = b.copy()
bCodes = []
for col in b2:
    typ = str(b2[col].dtype)
    if typ in ['int', 'int32', 'int64','float', 'float32', 'float64']:
        b2[col] = standardize(b2[col])
    elif typ in ['object', 'string']:
        b2[col], fcodes = cat_encode(b2[col])
        bCodes.append(fcodes)
        
b3 = b2.dropna().copy()
b3.reset_index(inplace=True, drop=True)

In [46]:
class Feature:
#Feature subclass for better use with KModes
    def __init__(self, featType, labels):
        #values in array form and feature dtype
        #self.vals = labels[~pd.isnull(labels)]
        #self.missing = np.where(pd.isnull(labels))[0]
        self.vals = labels
        self.size = self.vals.shape[0]
        self.type = featType
        #unique values, their orig indices and counts
        vals, inds, counts = np.unique(self.vals, return_inverse=True, return_counts=True)
        #Feature dictionary for use in itialization
        self.valDict = {v:c for v,c in  zip(vals, counts)}
        #frequency of values in orig indices
        self.valFreq = counts[inds]
        #self.valFreqMiss = np.insert(counts[inds], self.missing, 0)
        return
    
    def getCenter(self, cMsk):
    #Get center of feature in given centroid
        #Mean for num features
        #msk = cMsk[np.setdiff1d(np.arange(cMsk.shape[0]), self.missing)]
        msk = cMsk
        if self.type==float:
            return self.vals[msk].mean()
        #Mode for cat features
        else:
            return mode2(self.vals[msk], val=False)        
    
    def getDissimilarity(self, cntr, cMsk=None):
    #Given centroid center, get dissimilarity
        #Numerical feature use euclidean distance
        if self.type==float: 
            dissims = (self.vals - cntr)**2
        else:
            #Get Weight of points, from WHICH paper?
            weights = (self.valFreq + self.valDict[cntr])/(self.valFreq * self.valDict[cntr])
            #2007 dissimilarity
            if cMsk is not None:
                msk = cMsk
                #msk = cMsk[np.setdiff1d(np.arange(cMsk.shape[0]), self.missing)]
                dissims = weights * (1-(self.vals==cntr).astype(int) *((self.vals[msk] == cntr).sum()/msk.sum()))
            #Classic dissimilarity
            else:
                dissims = weights * (self.vals != cntr).astype(int)
        
        return dissims
        #dissims2 = np.insert(dissims, self.missing, 1000).astype(float)
        #return dissims2
        
    def calcEntropy(self, cMsk):
        msk = cMsk
        #msk = cMsk[np.setdiff1d(np.arange(cMsk.shape[0]), self.missing)]
        local_vals = self.vals[msk]
        if local_vals.shape[0] == 1: return 0
        
        keys, counts_raw = np.unique(local_vals, return_counts=True)        
        counts = counts_raw[counts_raw.nonzero()]
        probs = counts/counts.sum() 

        label_entropy = -np.sum(probs * np.log(probs)) 
        entropy_weight = 2*(1 - 1/(1 + np.exp(-label_entropy)))
        return entropy_weight * label_entropy

#######################################################################################################################

def initIndivCentroid(features, info, cntrd_id):
    cntrd_tmp = [0 for i in range(info['n_ftrs'])]

    for f, feat in enumerate(features):
        f_id = cntrd_id - np.sum(feat.missing < cntrd_id)
        if f_id in feat.missing:
            cntrd_tmp[f] = feat.getCenter(np.ones(feat.size, dtype=bool))
        else:
            cntrd_tmp[f] = feat.vals[f]

    return cntrd_tmp

def initCentroids(features, info):
#Init centroids based on Cao Paper
    #Empty list of centroids
    cntrds = [0 for i in range(info['n_clstrs'])]
    #Density of each point
    dens = np.zeros(info['n_rows'])
    #Iterate through features
    for feat in features:
        #Update point density
        dens += (feat.valFreq/(info['n_rows']*info['n_clstrs']))
    cntrds[0] = [feat.vals[dens.argmax()] for feat in features]
    #Iterate though remaining centroids, adapted from github implementation 
    for ki in range(1, info['n_clstrs']):
        adj_dens = np.zeros((ki, info['n_rows']))
        #Iterate through already created centroids
        for kii in range(0,ki):
            #Similarity btwn cntrds and pts
            dissims = np.zeros((info['n_rows'], info['n_ftrs']))
            #dissims = np.zeros(info['n_rows'])
            #Iterate through features
            for f, feat in enumerate(features):
                dissims[:,f] += feat.getDissimilarity(cntrds[kii][f]).astype(float)
            #Densities * similarites
            adj_dens[kii] = dens * dissims.sum(axis=1)
        #New centroid has max dens*sims from previous centroids
        cntrds[ki] = [feat.vals[np.argmax(np.min(adj_dens, axis=0))] for feat in features]
    #Assign intial membship
    cntrds, membship, cost = assignMembship(cntrds, features, info)
    return cntrds, membship, cost

def assignMembship(cntrds, features, info, membship=None, pred=True):
#Assign membship to centroids, membship=True if dissim is 2007, pred=True for KModes predict
    dissims = np.zeros((info['n_rows'], info['n_clstrs']))
    #Iterate through centroids
    for i, cntrd in enumerate(cntrds):
        #Get num and cat feat sims  
        num_feats = np.zeros((info['n_rows'], info['num']))
        cat_feats = np.zeros((info['n_rows'], info['cat']))
        n, c = 0, 0
        #Iterate thorugh features
        for f, feat in enumerate(features):
            #Num features
            if isinstance(feat.type, (float, int)):
                num_feats[:,n] += feat.getDissimilarity(cntrd[f]).astype(float)
                n += 1
            #Cat geatures
            else:
                #Choose which dissim to use
                if membship is not None:
                    #2007 dissim
                    cntrdMask = (membship == i)
                    cat_feats[:,c] += feat.getDissimilarity(cntrd[f], cntrdMask).astype(float)
                else:
                    #Orig disim
                    cat_feats[:,c] += feat.getDissimilarity(cntrd[f]).astype(float)
                c += 1
        #Sims of given centroid
        dissims[:,i] = num_feats.sum(axis=1) + info['gamma'] * cat_feats.sum(axis=1)
    print(dissims)
    #Choose smallest dissim for each pt
    membship = dissims.argmin(axis=1)
    #Cost is sum of smallest dissim
    cost = dissims.min(axis=1).sum()
    #If centroid has no assigned pts
    if (np.unique(membship).shape[0] != info['n_clstrs'])&(not pred):
        print('meep meep')
        cntrds = newCentroids(cntrds, features, membship, info)
        #Recursively assign membship
        cntrds, membship, cost = assignMemship(cntrds, features, info, membship)
    return cntrds, membship, cost

def newCentroids(cntrds, features, membship, info):
#Create new centroids if centroid is not present in membship
    #Missing centroids
    missing = np.setdiff1d(range(0,info['n_clstrs']), membship)
    #Largest centroid
    max_cntrd = mode2(membship, val=False)
    #Choose pts from largest centroid
    new_cntrds = np.random.choice(np.where(membship == max_cntrd)[0], missing.shape[0])
    #Replace old centroids with new points
    for m, nc in zip(missing, new_cntrds):
        cntrds[m] = [feat.vals[new_cntrds[nc]] for feat in features]
    return cntrds

def setCentroids(features, membship, info, dissim):
#Set centroids from center of each feature
    cntrds = [0 for c in range(info['n_clstrs'])]
    #Iterate through centroids, get approp center for each feature
    for i in range(info['n_clstrs']):
        cntrds[i] = [feat.getCenter(membship==i) for feat in features]
    #2007 dissim, pass membship
    if dissim:
        cntrds, membship, cost = assignMembship(cntrds, features, info, membship)
    #Classsic dissim
    else:
        cntrds, membship, cost = assignMembship(cntrds, features, info)
    return cntrds, membship, cost
 
def setMembshipToMatrix(membship, info):
#Convert membship to 2D array
    labels = np.zeros((info['n_rows'], info['n_clstrs']))
    for r, cls, in enumerate(membship):
        labels[r, cls] += 1
    return labels

def doGWE(features, msk):
    assigned = np.where(msk)[0]
    tot_e = np.sum([feat.calcEntropy(msk) for feat in features]) #calc initial weighted-entropy
    o_factors = np.array([tot_e for x in range(assigned.shape[0])])
    for o, i in enumerate(assigned):
        msk[i] = False
        wo = np.sum([feat.calcEntropy(msk) for feat in features])
        o_factors[o]  -= wo if wo != 0 else (tot_e + 1)
        msk[i] = True
    msk_msk = (o_factors > 0) 
    pts = np.zeros(msk.shape[0], dtype=bool)
    pts[assigned] = msk_msk
    return np.arange(msk.shape[0])[pts]

def getOutliers(features, membship, info):
    outliersList =  [0 for i in range(info['n_clstrs'])]
    
    for i in range(info['n_clstrs']):
        outliersList[i] = doGWE(features, membship == i)
        
    return np.array(list(itertools.chain.from_iterable(outliersList)))   


#######################################################################################################################

class KModes(BaseEstimator, ClusterMixin):
#Implementation of KModes, with help from github implementation
    def __init__(self, n_clusters=5, max_iter=100, gamma=1, dissim=False):
        #Keep all size information in dictionary
        self.info = {}
        self.info['n_clstrs'], self.info['max_itr'] = n_clusters, max_iter
        self.info['gamma'] = 1
        #2007 dissim or classic dissim
        self.dissim = dissim
        return     
    
    def fit(self, X, types):
    #Fit KModes to data
        #Convert Pandas to np array
        if 'pandas' in str(X.__class__):
            X = X.values
        
        self.info['n_rows'], self.info['n_ftrs'] = X.shape
        #Too few clusters
        assert self.info['n_clstrs'] <= self.info['n_rows']
        #Features to better form
        self.features = np.array([Feature(types[i], X[:,i]) for i in range(X.shape[1])])
        #Save n_ftrs and c_ftrs
        self.info['num'] = int(np.sum([1 for feat in self.features if isinstance(feat.type, (float, int))]))
        self.info['cat'] = int(self.info['n_ftrs'] - self.info['num'])
        
        #Init centroids, membship and cost
        cntrds, membship, cost = initCentroids(self.features, self.info)
    
        #Iterate until convergence or max iteration reached
        itr, converged = 0, False
        while (itr <= self.info['max_itr'])&(not converged):
            itr += 1
            #Set centroids, membship
            cntrds, new_membship, new_cost = setCentroids(self.features, membship, self.info, self.dissim)
            #Check if any pts changed membship
            chngs = np.sum(new_membship != membship)
            #Convergence conditions
            converged = (chngs == 0)|(new_cost >= cost)
            membship, cost = new_membship, new_cost
        
        #Save fit info
        self.outliers_ = getOutliers(self.features, membship, self.info)
        self.membship_ = membship
        self.matrix_ = setMembshipToMatrix(membship, self.info)
        self.centroids_, self.cost_, self.itr_ = cntrds, cost, itr
        return self   
    
    def predict(self, X):
    #Given new data, assignMembship to centroids
        assert hasattr(self, 'centroids_'), "Model not yet fit"
        if 'pandas' in str(X.__class__):
            X = X.values
        new_features = np.array([Feature(col.dtype, col) for col in X.T])
        new_info = self.info.copy()
        new_info[pts] = X.shape[0]
        cntrds, membship, cost = assignMembship(self.centroids_, new_features, new_info, pred=True)
        new_labels = setMembshipToLabels(membship, info)
        return new_labels, cost
    
    def fit_predict(self, X):
        return self.fit(X).predict(X)


In [47]:
a_types = [str, float, float, float, float, float, float, str, str, float, float, float, float, float, float, float, float, float, float, float, float, float]
ninjaClusters = KModes(n_clusters=int(np.sqrt(a3.shape[0]/2))).fit(a3, a_types)

19
8
38
[[  2.48842272 151.32468178  13.86657714 121.4746843 ]
 [  2.70333214 147.80107104  13.02143565 118.61745511]
 [  2.17685113 157.95578496  14.20836242 127.74155123]
 [  2.18053985 157.71079566  13.75834073 128.10302394]
 [  2.93269639 148.98953356  13.69633983 120.52640544]
 [  2.56625261 151.01197772  13.98325342 120.76364646]
 [  2.58059224 150.28887025  13.02184666 121.18617171]
 [  2.85887613 146.75468937  13.81631583 117.50808908]
 [ 14.96054916 122.52854124   0.         116.32765816]
 [  2.59990748 149.10704952  11.59375142 121.64207088]
 [  2.34333135 154.66303416  13.58157694 125.23181979]
 [  3.01047317 146.64871533  14.02548975 117.71046557]
 [  2.0469968  163.18101235  14.36878539 132.90735374]
 [  2.77680228 148.66644992  13.74549989 118.75387421]
 [  2.64781468 149.22820375  14.05840029 119.43407077]
 [  2.63912435 150.47261583  14.17564919 120.46451022]
 [  0.         167.35981965  14.96054916 136.7165307 ]
 [  2.33178926 154.15738982  13.79456136 124.35916018]
 [

In [168]:
a3

In [54]:
b_types = [float, str, float, float, str, float, float, float, float, float, float, float, float, str, float, float]
h3zmClusters = KModes(n_clusters=int(np.sqrt(b2.shape[0]/2))).fit(b2, b_types)

In [183]:
ninjaClusters.outliers_

array([39, 40])

In [55]:
h3zmClusters.outliers_

array([152, 166, 248, 262, 264, 346, 444, 458, 183, 185, 191, 485, 187,
        72,  74, 154, 464, 466, 104, 202, 300, 398])

In [12]:
np.warnings.filterwarnings('ignore')

In [122]:
class Feature:
#Feature subclass for better use with KModes
    def __init__(self, featType, labels):
        #values in array form and feature dtype
        self.vals = labels[~pd.isnull(labels)]
        self.missing = np.where(pd.isnull(labels))[0]
        self.size = self.vals.shape[0]
        self.type = featType
        #unique values, their orig indices and counts
        vals, inds, counts = np.unique(self.vals, return_inverse=True, return_counts=True)
        #Feature dictionary for use in itialization
        self.valDict = {v:c for v,c in  zip(vals, counts)}
        #frequency of values in orig indices
        self.valFreq = counts[inds]
        self.valFreqMiss = self.fillMissing(self.valFreq)
        return
    
    def fillMissing(self, arr, high=False):
        for miss in self.missing:
            if high:
                arr = np.concatenate((arr[:miss], [np.nan], arr[miss:]))
            else:
                arr = np.concatenate((arr[:miss], [np.nan], arr[miss:]))
        return arr
    
    def getCenter(self, cMsk):
    #Get center of feature in given centroid
        #Mean for num features
        msk = cMsk[np.setdiff1d(np.arange(cMsk.shape[0]), self.missing)]
        if self.type==float:
            return self.vals[msk].mean()
        #Mode for cat features
        else:
            return mode2(self.vals[msk], val=False)        
    
    def getDissimilarity(self, cntr, high=False, cMsk=None):
    #Given centroid center, get dissimilarity
        #Numerical feature use euclidean distance
        if self.type==float: 
            dissims = (self.vals - cntr)**2
        else:
            #Get Weight of points, from WHICH paper?
            weights = (self.valFreq + self.valDict[cntr])/(self.valFreq * self.valDict[cntr])
            #2007 dissimilarity
            if cMsk is not None:
                msk = cMsk[np.setdiff1d(np.arange(cMsk.shape[0]), self.missing)]
                dissims = weights * (1-(self.vals==cntr).astype(int) *((self.vals[msk] == cntr).sum()/msk.sum()))
            #Classic dissimilarity
            else:
                dissims = weights * (self.vals != cntr).astype(int)
                
        dissims = self.fillMissing(dissims, high).astype(np.float64)
        return dissims
                
    def calcEntropy(self, cMsk):
        msk = cMsk[np.setdiff1d(np.arange(cMsk.shape[0]), self.missing)]
        local_vals = self.vals[msk]
        if local_vals.shape[0] == 1: return 0
        
        keys, counts_raw = np.unique(local_vals, return_counts=True)        
        counts = counts_raw[counts_raw.nonzero()]
        probs = counts/counts.sum() 

        label_entropy = -np.sum(probs * np.log(probs)) 
        entropy_weight = 2*(1 - 1/(1 + np.exp(-label_entropy)))
        return entropy_weight * label_entropy

#######################################################################################################################

def initCentroids(features, info):
#Init centroids based on Cao Paper
    #Empty list of centroids
    cntrds = [0 for i in range(info['n_clstrs'])]
    #Density of each point
    dens = np.zeros(info['n_rows'])
    #Iterate through features
    for feat in features:
        #Update point density
        dens += (feat.valFreqMiss/(info['n_rows']*info['n_clstrs']))
    #cntrds[0] = [feat.vals[dens.argmax()] for feat in features]
    cntrds[0] = [feat.vals[np.nanargmax(dens)] for feat in features]
    #Iterate though remaining centroids, adapted from github implementation 
    for ki in range(1, info['n_clstrs']):
        adj_dens = np.zeros((ki, info['n_rows']))
        #Iterate through already created centroids
        for kii in range(0,ki):
            #Similarity btwn cntrds and pts
            dissims = np.zeros((info['n_rows'], info['n_ftrs']))
            #dissims = np.zeros(info['n_rows'])
            #Iterate through features
            for f, feat in enumerate(features):
                dissims[:,f] += feat.getDissimilarity(cntrds[kii][f], high=False)
            #Densities * similarites
            adj_dens[kii] = (dens * dissims.sum(axis=1)).astype(np.float64)
        #New centroid has max dens*sims from previous centroids
        cntrds[ki] = [feat.vals[np.nanargmax(np.nanmin(adj_dens.astype(np.float64), axis=0))] for feat in features]
    #Assign intial membship
    cntrds, membship, cost = assignMembship(cntrds, features, info)
    return cntrds, membship, cost

def assignMembship(cntrds, features, info, membship=None, pred=True):
#Assign membship to centroids, membship=True if dissim is 2007, pred=True for KModes predict
    dissims = np.zeros((info['n_rows'], info['n_clstrs']))
    #Iterate through centroids
    for i, cntrd in enumerate(cntrds):
        #Get num and cat feat sims  
        num_feats = np.zeros((info['n_rows'], info['num']))
        cat_feats = np.zeros((info['n_rows'], info['cat']))
        n, c = 0, 0
        #Iterate thorugh features
        for f, feat in enumerate(features):
            #Num features
            if isinstance(feat.type, (float, int)):
                num_feats[:,n] += feat.getDissimilarity(cntrd[f], high=True)
                n += 1
            #Cat geatures
            else:
                #Choose which dissim to use
                if membship is not None:
                    #2007 dissim
                    cntrdMask = (membship == i)
                    cat_feats[:,c] += feat.getDissimilarity(cntrd[f], high=True, cMsk=cntrdMask)
                else:
                    #Orig disim
                    cat_feats[:,c] += feat.getDissimilarity(cntrd[f], high=True)
                c += 1
        #Sims of given centroid
        dissims[:,i] = np.nansum(num_feats, axis=1) + info['gamma'] * np.nansum(cat_feats, axis=1)
    #Choose smallest dissim for each pt
    membship = np.nanargmin(dissims, axis=1)
    #Cost is sum of smallest dissim
    cost = np.nanmin(dissims, axis=1).sum()
    #If centroid has no assigned pts
    if (np.unique(membship).shape[0] != info['n_clstrs'])&(not pred):
        print('meep meep')
        cntrds = newCentroids(cntrds, features, membship, info)
        #Recursively assign membship
        cntrds, membship, cost = assignMemship(cntrds, features, info, membship)
    return cntrds, membship, cost

def newCentroids(cntrds, features, membship, info):
#Create new centroids if centroid is not present in membship
    #Missing centroids
    missing = np.setdiff1d(range(0,info['n_clstrs']), membship)
    #Largest centroid
    max_cntrd = mode2(membship, val=False)
    #Choose pts from largest centroid
    new_cntrds = np.random.choice(np.where(membship == max_cntrd)[0], missing.shape[0])
    #Replace old centroids with new points
    for m, nc in zip(missing, new_cntrds):
        cntrds[m] = [feat.vals[new_cntrds[nc]] for feat in features]
    return cntrds

def setCentroids(features, membship, info, dissim):
#Set centroids from center of each feature
    cntrds = [0 for c in range(info['n_clstrs'])]
    #Iterate through centroids, get approp center for each feature
    for i in range(info['n_clstrs']):
        cntrds[i] = [feat.getCenter(membship==i) for feat in features]
    #2007 dissim, pass membship
    if dissim:
        cntrds, membship, cost = assignMembship(cntrds, features, info, membship)
    #Classsic dissim
    else:
        cntrds, membship, cost = assignMembship(cntrds, features, info)
    return cntrds, membship, cost
 
def setMembshipToMatrix(membship, info):
#Convert membship to 2D array
    labels = np.zeros((info['n_rows'], info['n_clstrs']))
    for r, cls, in enumerate(membship):
        labels[r, cls] += 1
    return labels

def doGWE(features, msk):
    assigned = np.where(msk)[0]
    tot_e = np.sum([feat.calcEntropy(msk) for feat in features]) #calc initial weighted-entropy
    o_factors = np.array([tot_e for x in range(assigned.shape[0])])
    for o, i in enumerate(assigned):
        msk[i] = False
        wo = np.sum([feat.calcEntropy(msk) for feat in features])
        o_factors[o]  -= wo if wo != 0 else (tot_e + 1)
        msk[i] = True
    msk_msk = (o_factors > 0) 
    pts = np.zeros(msk.shape[0], dtype=bool)
    pts[assigned] = msk_msk
    return np.arange(msk.shape[0])[pts]

def getOutliers(features, membship, info):
    outliersList =  [0 for i in range(info['n_clstrs'])]
    
    for i in range(info['n_clstrs']):
        outliersList[i] = doGWE(features, membship == i)
        
    return np.array(list(itertools.chain.from_iterable(outliersList)))   


#######################################################################################################################

class KModes(BaseEstimator, ClusterMixin):
#Implementation of KModes, with help from github implementation
    def __init__(self, n_clusters=5, max_iter=100, gamma=1, dissim=False):
        #Keep all size information in dictionary
        self.info = {}
        self.info['n_clstrs'], self.info['max_itr'] = n_clusters, max_iter
        self.info['gamma'] = 1
        #2007 dissim or classic dissim
        self.dissim = dissim
        return     
    
    def fit(self, X, types):
    #Fit KModes to data
        #Convert Pandas to np array
        if 'pandas' in str(X.__class__):
            X = X.values
        
        self.info['n_rows'], self.info['n_ftrs'] = X.shape
        #Too few clusters
        assert self.info['n_clstrs'] <= self.info['n_rows']
        #Features to better form
        self.features = np.array([Feature(types[i], X[:,i]) for i in range(X.shape[1])])
        #Save n_ftrs and c_ftrs
        self.info['num'] = int(np.sum([1 for feat in self.features if isinstance(feat.type, (float, int))]))
        self.info['cat'] = int(self.info['n_ftrs'] - self.info['num'])
        
        #Init centroids, membship and cost
        cntrds, membship, cost = initCentroids(self.features, self.info)
    
        #Iterate until convergence or max iteration reached
        itr, converged = 0, False
        while (itr <= self.info['max_itr'])&(not converged):
            itr += 1
            #Set centroids, membship
            cntrds, new_membship, new_cost = setCentroids(self.features, membship, self.info, self.dissim)
            #Check if any pts changed membship
            chngs = np.sum(new_membship != membship)
            #Convergence conditions
            converged = (chngs == 0)|(new_cost >= cost)
            membship, cost = new_membship, new_cost
        
        #Save fit info
        self.outliers_ = getOutliers(self.features, membship, self.info)
        self.membship_ = membship
        self.matrix_ = setMembshipToMatrix(membship, self.info)
        self.centroids_, self.cost_, self.itr_ = cntrds, cost, itr
        return self   
    
    def predict(self, X, types):
    #Given new data, assignMembship to centroids
        assert hasattr(self, 'centroids_'), "Model not yet fit"
        if 'pandas' in str(X.__class__):
            X = X.values
        new_features = np.array([Feature(types[i], X[:,i]) for i in range(X.shape[1])])
        new_info = self.info.copy()
        new_info['n_rows'] = X.shape[0]
        cntrds, membship, cost = assignMembship(self.centroids_, new_features, new_info, pred=True)
        new_labels = setMembshipToMatrix(membship, new_info)
        return new_labels, cost
    
    def fit_predict(self, X, types):
        return self.fit(X, types).predict(X, types)


In [123]:
a_types = [str, float, float, float, float, float, float, str, str, float, float, float, float, float, float, float, float, float, float, float, float, float]
ninjaClusters = KModes(n_clusters=int(np.sqrt(a2.shape[0]/2))).fit(a2, a_types)

In [101]:
b_types = [float, str, float, float, str, float, float, float, float, float, float, float, float, str, float, float]
h3zmClusters = KModes(n_clusters=int(np.sqrt(b2.shape[0]/2))).fit(b2, b_types)

In [125]:
ninjaClusters.itr_

2